In [1]:
import pandas as pd 
import numpy as np
import math
import keras
import tensorflow as tf
print(pd.__version__)
import progressbar
import os
from os import listdir

1.2.3


## Print Dependencies



Dependences are fundamental to record the computational environment.

In [2]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,keras,numpy,math,tensorflow,matplotlib,h5py

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.21.0

pandas    : 1.2.3
keras     : 2.4.3
numpy     : 1.19.5
math      : unknown
tensorflow: 2.4.1
matplotlib: 3.3.4
h5py      : 2.10.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.8.0-48-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

 
Last updated: Tue Mar 30 2021 09:57:56CEST



## Load of the data

   You can also load all of them! Writing "all_data"

In [3]:
from process import loaddata
class_data = loaddata("../data/{}.csv".format('probability'))

In [4]:
class_data = class_data[class_data[:,0] > 0.0001]

In [5]:
np.random.shuffle(class_data)
y = class_data[:,0]
#y2 = class_data[:,-1]
x = class_data[:,1:14]

In [6]:
#y = y1/y2
#y = np.squeeze(np.dstack((y1,y2))) 

In [7]:
#print(y2.shape)
#print(y1.shape)
#print(y.shape)

In [8]:
x.shape

(135911, 13)

In [9]:
train_split = 0.75
train_limit = int(len(y)*train_split)
print("Training sample: {0} \nValuation sample: {1}".format(train_limit, len(y)-train_limit))

Training sample: 101933 
Valuation sample: 33978


In [10]:
x_train = x[:train_limit]
x_val = x[train_limit:]

y_train = y[:train_limit]
y_val = y[train_limit:]

## Model Build

In [11]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from keras.layers.normalization import BatchNormalization

In [26]:
def mase(y_true, y_pred):

    sust = tf.reduce_mean(tf.abs(y_true[:,1:] - y_true[:,:-1]))
    diff = tf.reduce_mean(tf.abs(y_pred - y_true))

    return diff/sust

In [31]:
def build_model() :
    model = models.Sequential()
    model.add (BatchNormalization(input_dim = 13))
    model.add (layers.Dense (16 , activation = "sigmoid"))
    model.add (layers.Dense (32, activation = "relu"))
    model.add (layers.Dense (64 , activation = "relu"))
    model.add (layers.Dense (128 , activation = "relu"))
    model.add (layers.Dense (64, activation = "relu"))
    model.add (layers.Dense (32 , activation = "relu"))
    model.add (layers.Dense (16, activation = "relu"))
    model.add (layers.Dense (12 , activation = "relu"))
    model.add (layers.Dense (1 , activation = "sigmoid"))
    model.compile(optimizer = "adam" , loss = "rmse" , metrics = ["mae"])
    return model

In [32]:
model = build_model ()
history = model.fit ( x_train, y_train, epochs = 10000, batch_size = 50000 , validation_data = (x_val, y_val) )
model.save("../models/classifier/{}_nodropout.h5".format('probability'))

Epoch 1/10000


ValueError: in user code:

    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/chiara/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

accuracy = history.history['mae']
val_accuracy = history.history['val_mae']


epochs = range(1, len(loss) + 1)
fig, ax1 = plt.subplots()

l1 = ax1.plot(epochs, loss, 'bo', label='Training loss')
vl1 = ax1.plot(epochs, val_loss, 'b', label='Validation loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss (mae))')

ax2 = ax1.twinx()
ac2= ax2.plot(epochs, accuracy, 'o', c="red", label='Training acc')
vac2= ax2.plot(epochs, val_accuracy, 'r', label='Validation acc')
ax2.set_ylabel('mape')

lns = l1 + vl1 + ac2 + vac2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc="center right")
fig.tight_layout()
#fig.savefig("acc+loss_drop.pdf")
fig.show()

In [ ]:
plt.clf()   # clear figure

accuracy = history.history['mae']
val_accuracy = history.history['val_mae']

plt.plot(epochs, accuracy, 'bo', label='Training acc')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Probability density distribution

In [ ]:
y

In [ ]:
y_pred = model.predict(x)

In [ ]:
y_pred

In [ ]:
def probability(prob_array, bins):
    prob_array = np.array(prob_array)
    plt.hist(prob_array, bins, histtype=u'step', density=True)
    plt.show()

In [ ]:
from matplotlib import pyplot
y = np.array(y)
bins = np.linspace(0, 0.6, 100)
pyplot.hist(y, bins, color = 'indianred', alpha=0.5, label='Osiris')
pyplot.hist(y_pred, bins, color = 'mediumslateblue', alpha=0.5, label='NN')
pyplot.legend(loc='upper right')
pyplot.xlabel('Probability')
pyplot.title('Trained on ($p_e$, $p_{\gamma}$, $x_e$, $x_{\gamma}$, $\omega_e$, $\omega_{\gamma}$, n)')
pyplot.show()